## Imports e configs

In [1]:
# imports
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
import warnings

# pandas config
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100
pd.options.display.float_format = '{:,.2f}'.format
warnings.filterwarnings('ignore')

# seed reproducibilidade
myseed = 484

## Abrindo os dados

In [2]:
# pegando os dados do github e aprindo o json
data_url = "https://github.com/sthemonica/alura-voz/blob/main/Dados/Telco-Customer-Churn.json?raw=true"
response = urlopen(data_url)
data_json = json.loads(response.read())

# carregando o dataframe
df = pd.json_normalize(data_json, max_level=2, sep='_')

# removendo o prefixo
df.columns = ['_'.join(cols.split('_')[1:]) if len(cols.split('_')) > 1 else cols for cols in df.columns]

# mostrando o cabecalho
df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4


## Entendendo as Colunas
Dicionario de informacao das colunas

In [3]:
# dicionario informacao das colunas
dict_info_cols = { 
    'customerID': 'número de identificação único de cada cliente',
    'Churn': 'se o cliente deixou ou não a empresa',
    'gender': 'gênero (masculino e feminino)' ,
    'SeniorCitizen': 'informação sobre um cliente ter ou não idade igual ou maior que 65 anos',
    'Partner':  'se o cliente possui ou não um parceiro ou parceira',
    'Dependents': 'se o cliente possui ou não dependentes',
    'tenure':  'meses de contrato do cliente',
    'PhoneService': 'assinatura de serviço telefônico' ,
    'MultipleLines': 'assisnatura de mais de uma linha de telefone' ,
    'InternetService': 'assinatura de um provedor internet' ,
    'OnlineSecurity': 'assinatura adicional de segurança online' ,
    'OnlineBackup': 'assinatura adicional de backup online' ,
    'DeviceProtection': 'assinatura adicional de proteção no dispositivo' ,
    'TechSupport': 'assinatura adicional de suporte técnico, menos tempo de espera',
    'StreamingTV': 'assinatura de TV a cabo',
    'StreamingMovies': 'assinatura de streaming de filmes' ,
    'Contract': 'tipo de contrato',
    'PaperlessBilling': 'se o cliente prefere receber online a fatura',
    'PaymentMethod': 'forma de pagamento',
    'Charges_Monthly': 'total de todos os serviços do cliente por mês',
    'Charges_Total': 'total gasto pelo cliente'
    }

Pela descrição das colunas podemos classificar o tipo de variavel entre qualitativos e quantitativos.

|Abreviacao|Significado|  
|--|--|
| **QT** | Quantitativo |
| **QL** | Qualitativo |
| **ID** | Identificacao |

In [4]:
# classificando colunas em (identificao, qualitativos e quantitativos)
coluna_id = ['customerID']
colunas_quantitativas = ['tenure', 'Charges_Monthly', 'Charges_Total']
colunas_qualitativas = df.drop(colunas_quantitativas + coluna_id, axis=1).columns.tolist()

In [5]:
info = pd.DataFrame({'name' : df.columns})
# classifica (qualitativa, quantitativa, identificacao)
info['class_var'] = info['name'].replace(colunas_qualitativas, 'QL').replace(
    colunas_quantitativas, 'QT').replace(coluna_id, 'ID' )
info['information'] = info['name'].map(dict_info_cols)
info['dict_cat'] = ['-' for _ in info['name']]
info


,name,class_var,information,dict_cat
0,customerID,ID,número de identificação único de cada cliente,-
1,Churn,QL,se o cliente deixou ou não a empresa,-
2,gender,QL,gênero (masculino e feminino),-
3,SeniorCitizen,QL,informação sobre um cliente ter ou não idade igual ou maior que 65 anos,-
4,Partner,QL,se o cliente possui ou não um parceiro ou parceira,-
5,Dependents,QL,se o cliente possui ou não dependentes,-
6,tenure,QT,meses de contrato do cliente,-
7,PhoneService,QL,assinatura de serviço telefônico,-
8,MultipleLines,QL,assisnatura de mais de uma linha de telefone,-
9,InternetService,QL,assinatura de um provedor internet,-


## DataFrame de Verificacao de Dados

Funcao para criacao do df check

In [6]:
def df_check(df):
    
    # criando dataframe para verificacao
    check = pd.DataFrame({'name': df.columns})

    # verificando quantos valores unicos existem em cada  (numero maximo possivel 7267)
    check['nunique'] = df.nunique().values

    # verificando quantidades de nulos
    check['isnull'] = df.isnull().sum().values

    # verificando espacos faltantes
    temp_series1 = df[df.select_dtypes('object').columns].apply(lambda x: x.str.strip().isin(['']).sum()) # colunas str
    temp_series2 = df.select_dtypes(['float', 'integer']).isnull().sum() # colunas numericos
    temp_series = pd.concat([temp_series1, temp_series2])
    temp_series.name = 'blank'
    check = check.merge(temp_series, how='left', left_on='name', right_index=True)

    # verificando data type
    check['dtypes'] = df.dtypes.values

    # vericando quais os valores unicos para colunas com 5 ou menos valores unicos
    check['unique'] = df.apply(lambda x: x.unique() if x.nunique() <= 5 else '-').values

    return check

In [7]:
check = df_check(df)
check

,name,nunique,isnull,blank,dtypes,unique
0,customerID,7267,0,0,object,-
1,Churn,3,0,224,object,"[No, Yes, ]"
2,gender,2,0,0,object,"[Female, Male]"
3,SeniorCitizen,2,0,0,int64,"[0, 1]"
4,Partner,2,0,0,object,"[Yes, No]"
5,Dependents,2,0,0,object,"[Yes, No]"
6,tenure,73,0,0,int64,-
7,PhoneService,2,0,0,object,"[Yes, No]"
8,MultipleLines,3,0,0,object,"[No, Yes, No phone service]"
9,InternetService,3,0,0,object,"[DSL, Fiber optic, No]"


Aqui eh possivel notar que 2 colunas do data frama apresentam dados em branco ('blank'), as colunas *Churn* e *Charges_Total*

#### Churn

In [8]:
df.Churn.value_counts(normalize=True)*100

No    71.20
Yes   25.72
       3.08
Name: Churn, dtype: float64

In [9]:
df.Churn.unique()

array(['No', 'Yes', ''], dtype=object)

In [10]:
df[df.Churn == ''].sample(20, random_state=myseed).corr()

,SeniorCitizen,tenure,Charges_Monthly
SeniorCitizen,1.00,0.07,0.28
tenure,0.07,1.00,0.15
Charges_Monthly,0.28,0.15,1.00


Existem 3.08% de dados que estao vazios, como representa uma pequena parte do conjunto eh possivel fazer a remocao das linhas no tratamento, mas gostaria de investigar se a falta da informacao tem alguma correlacao com outra variavel.

#### Charges_Total

In [11]:
df.Charges_Total.value_counts().head(10)

         11
20.2     11
19.75     9
19.55     9
19.9      9
19.65     8
20.05     8
45.3      7
20.25     7
20.3      6
Name: Charges_Total, dtype: int64

Temos um dado vazio, que aparecem 11 vezes dentro do nosso conjunto de dados

In [12]:
valor_vazio = df.Charges_Total.value_counts().index[0]
valor_vazio

' '

In [13]:
indices = df.query(f'Charges_Total == "{valor_vazio}"').index
df.loc[indices, colunas_quantitativas]

,tenure,Charges_Monthly,Charges_Total
975,0,56.05,
1775,0,20.00,
1955,0,61.90,
2075,0,19.70,
2232,0,20.25,
2308,0,25.35,
2930,0,73.35,
3134,0,25.75,
3203,0,52.55,
4169,0,80.85,


Com uma breve analise podemos perceber que o motivo do valor em branco eh o periodo de contrato (a coluna tenure). 
Para o tratamento, vou substituir esse valores vazios por zero.

In [14]:
df.Charges_Total.replace(valor_vazio, 0, inplace=True)
df['Charges_Total'] = pd.to_numeric(df['Charges_Total'])
df.loc[indices, colunas_quantitativas]

,tenure,Charges_Monthly,Charges_Total
975,0,56.05,0.00
1775,0,20.00,0.00
1955,0,61.90,0.00
2075,0,19.70,0.00
2232,0,20.25,0.00
2308,0,25.35,0.00
2930,0,73.35,0.00
3134,0,25.75,0.00
3203,0,52.55,0.00
4169,0,80.85,0.00


## Trabalhando nas colunas qualitativas

In [15]:
info.query('class_var == "QL"')

,name,class_var,information,dict_cat
1,Churn,QL,se o cliente deixou ou não a empresa,-
2,gender,QL,gênero (masculino e feminino),-
3,SeniorCitizen,QL,informação sobre um cliente ter ou não idade igual ou maior que 65 anos,-
4,Partner,QL,se o cliente possui ou não um parceiro ou parceira,-
5,Dependents,QL,se o cliente possui ou não dependentes,-
7,PhoneService,QL,assinatura de serviço telefônico,-
8,MultipleLines,QL,assisnatura de mais de uma linha de telefone,-
9,InternetService,QL,assinatura de um provedor internet,-
10,OnlineSecurity,QL,assinatura adicional de segurança online,-
11,OnlineBackup,QL,assinatura adicional de backup online,-


In [16]:
dict_dicts = {
    'Churn' : {
        'No': 0,
        'Yes': 1,
        '': -1 # Sem informacao
    },
    'gender': {
        'Female': 0,
        'Male': 1
    },
    'SeniorCitizen': {
        0: 0, # menos de 65
        1: 1 # 65 ou mais
    },
    'Partner': {
        'Yes': 1,
        'No': 0
    },
    'Dependents': {
        'Yes': 1,
        'No': 0
    },
    'PhoneService': {
        'Yes': 1,
        'No': 0
    },
    'MultipleLines': {
        'Yes': 1,
        'No': 0,
        'No phone service' : -1
    },
    'InternetService': {
        'DSL': 1,
        'Fiber optic': 2,
        'No': 0
    },
    'OnlineSecurity': {
        'No': 0,
        'Yes': 1,
        'No internet service': -1
    },
    'OnlineBackup': {
        'Yes': 1,
        'No': 0, 
        'No internet service': -1
    },
    'DeviceProtection': {
        'Yes': 1,
        'No': 0, 
        'No internet service': -1
    },
    'TechSupport': {
        'Yes': 1,
        'No': 0, 
        'No internet service': -1
    },
    'StreamingTV': {
        'Yes': 1,
        'No': 0, 
        'No internet service': -1
    },
    'StreamingMovies': {
        'Yes': 1,
        'No': 0, 
        'No internet service': -1
    },
    'Contract': {
        'One year': 12,
        'Month-to-month': 1,
        'Two year': 24
    },
    'PaperlessBilling': {
        'Yes': 1,
        'No': 0
    },
    'PaymentMethod': {
        'Mailed check':0,
        'Electronic check':1,
        'Credit card (automatic)':2,
        'Bank transfer (automatic)':3
    }
}
info['dict_cat'] = info['name'].map(dict_dicts).fillna('-')
info

,name,class_var,information,dict_cat
0,customerID,ID,número de identificação único de cada cliente,-
1,Churn,QL,se o cliente deixou ou não a empresa,"{'No': 0, 'Yes': 1, '': -1}"
2,gender,QL,gênero (masculino e feminino),"{'Female': 0, 'Male': 1}"
3,SeniorCitizen,QL,informação sobre um cliente ter ou não idade igual ou maior que 65 anos,"{0: 0, 1: 1}"
4,Partner,QL,se o cliente possui ou não um parceiro ou parceira,"{'Yes': 1, 'No': 0}"
5,Dependents,QL,se o cliente possui ou não dependentes,"{'Yes': 1, 'No': 0}"
6,tenure,QT,meses de contrato do cliente,-
7,PhoneService,QL,assinatura de serviço telefônico,"{'Yes': 1, 'No': 0}"
8,MultipleLines,QL,assisnatura de mais de uma linha de telefone,"{'Yes': 1, 'No': 0, 'No phone service': -1}"
9,InternetService,QL,assinatura de um provedor internet,"{'DSL': 1, 'Fiber optic': 2, 'No': 0}"


In [17]:
df_tratado = df.copy()

In [18]:
for key in dict_dicts:
    df_tratado[key] = df_tratado[key].map(dict_dicts[key])
df_tratado

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,0,0,0,1,1,9,1,0,1,0,1,0,1,1,0,12,1,0,65.60,593.30
1,0003-MKNFE,0,1,0,0,0,9,1,1,1,0,0,0,0,0,1,1,0,0,59.90,542.40
2,0004-TLHLJ,1,1,0,0,0,4,1,0,2,0,0,1,0,0,0,1,1,1,73.90,280.85
3,0011-IGKFF,1,1,1,1,0,13,1,0,2,0,1,1,0,1,1,1,1,1,98.00,"1,237.85"
4,0013-EXCHZ,1,0,1,1,0,3,1,0,2,0,0,0,1,1,0,1,1,0,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,0,0,0,0,0,13,1,0,1,1,0,0,1,0,0,12,0,0,55.15,742.90
7263,9992-RRAMN,1,1,0,1,0,22,1,1,2,0,0,0,0,0,1,1,1,1,85.10,"1,873.70"
7264,9992-UJOEL,0,1,0,0,0,2,1,0,1,0,1,0,0,0,0,1,1,0,50.30,92.75
7265,9993-LHIEB,0,1,0,1,1,67,1,0,1,1,0,1,1,0,1,24,0,0,67.85,"4,627.65"


In [19]:
df_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   int64  
 2   gender            7267 non-null   int64  
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   int64  
 5   Dependents        7267 non-null   int64  
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   int64  
 8   MultipleLines     7267 non-null   int64  
 9   InternetService   7267 non-null   int64  
 10  OnlineSecurity    7267 non-null   int64  
 11  OnlineBackup      7267 non-null   int64  
 12  DeviceProtection  7267 non-null   int64  
 13  TechSupport       7267 non-null   int64  
 14  StreamingTV       7267 non-null   int64  
 15  StreamingMovies   7267 non-null   int64  
 16  Contract          7267 non-null   int64  


In [20]:
df_check(df_tratado)

,name,nunique,isnull,blank,dtypes,unique
0,customerID,7267,0,0,object,-
1,Churn,3,0,0,int64,"[0, 1, -1]"
2,gender,2,0,0,int64,"[0, 1]"
3,SeniorCitizen,2,0,0,int64,"[0, 1]"
4,Partner,2,0,0,int64,"[1, 0]"
5,Dependents,2,0,0,int64,"[1, 0]"
6,tenure,73,0,0,int64,-
7,PhoneService,2,0,0,int64,"[1, 0]"
8,MultipleLines,3,0,0,int64,"[0, 1, -1]"
9,InternetService,3,0,0,int64,"[1, 2, 0]"


In [21]:
df_tratado.corrwith(df_tratado['Churn']).sort_values()

Contract           -0.34
tenure             -0.31
Charges_Total      -0.17
Dependents         -0.15
Partner            -0.14
PaymentMethod      -0.09
gender             -0.01
PhoneService        0.01
OnlineSecurity      0.03
TechSupport         0.03
MultipleLines       0.03
OnlineBackup        0.07
DeviceProtection    0.08
SeniorCitizen       0.13
StreamingMovies     0.15
StreamingTV         0.15
PaperlessBilling    0.16
Charges_Monthly     0.17
InternetService     0.28
Churn               1.00
dtype: float64

## Trabalhando nas colunas quantitativas

In [22]:
df_tratado[colunas_quantitativas].describe()

,tenure,Charges_Monthly,Charges_Total
count,"7,267.00","7,267.00","7,267.00"
mean,32.35,64.72,"2,277.18"
std,24.57,30.13,"2,268.65"
min,0.00,18.25,0.00
25%,9.00,35.42,396.20
50%,29.00,70.30,"1,389.20"
75%,55.00,89.88,"3,778.52"
max,72.00,118.75,"8,684.80"


In [23]:
cols_qtt = df_tratado[colunas_quantitativas]
cols_qtt['Charges_Total_Calculated'] = cols_qtt['Charges_Monthly'] * cols_qtt['tenure']

In [24]:
cols_qtt.describe()

,tenure,Charges_Monthly,Charges_Total,Charges_Total_Calculated
count,"7,267.00","7,267.00","7,267.00","7,267.00"
mean,32.35,64.72,"2,277.18","2,277.03"
std,24.57,30.13,"2,268.65","2,266.54"
min,0.00,18.25,0.00,0.00
25%,9.00,35.42,396.20,393.90
50%,29.00,70.30,"1,389.20","1,389.50"
75%,55.00,89.88,"3,778.52","3,782.45"
max,72.00,118.75,"8,684.80","8,550.00"
